# Correlations with Repair Genes

Here we will see if our event correlates with any of the following known DNA repair genes:

* **TP53** tumor suppressor, regulates cell division
* **KMT2C** histone methyltransferase, regulates gene transcription by modifying chromatin structure
* **ATM** responds to DNA damage by phosphorylating key substrates involved in DNA repair
* **BAP1** Binds BRCA1, deubiquitinating enzyme
* **BRCA2** deficiency known to impair completion of cell division by cytokinesis
* **POLQ** contains RAD51 binding motifs, blocks RAD51-mediated recombination
* **PRKDC** involved in DNA NHEJ during DNA double-strand break repair
* **ATR** regulator of genomic integrity, controls and coordinates DNA-replication origin firing, replication-fork stability, cell-cycle checkpoints, and DNA repair
* **BRCA1** critical roles in DNA repair, cell cycle checkpoint control, and maintenance of genomic stability
* **MSH6** part of a large multisubunit protein complex of tumor suppressors
* **POLE** catalytic subunit of DNA polymerase epsilon, involved in DNA repair
* **FANCM** direct role in DNA processing
* **TP53BP1** binds to WT p53, but not mutatant p53, also regulates ATM-dependent phosphorylation events
* **CDK12** forms protein complexes with cyclin K, regulates expression of long complex genes, several of which have roles in DNA maintenance and repair
* **FANCD2** links FA and ATM damage response pathways
* **SLX4** critical scaffold element for the assembly of a multiprotein complex containing enzymes involved in DNA maintenance and repair

## Setup

In [1]:
import cptac
import pandas as pd
import numpy as np
import scipy.stats as stats
import statsmodels.stats.multitest

cptac warning: Your version of cptac (0.8.5) is out-of-date. Latest is 0.8.6. Please run 'pip install --upgrade cptac' to update it. (C:\Users\chels\anaconda3\envs\notebooks\lib\site-packages\ipykernel_launcher.py, line 1)


In [2]:
EVENT_COLUMN = 'loss_event'

In [3]:
cancer_types = {
    'brca': cptac.Brca(),
    'ccrcc': cptac.Ccrcc(),
    'colon': cptac.Colon(),
    'endo': cptac.Endometrial(),
    'gbm': cptac.Gbm(),
    'hnscc': cptac.Hnscc(),
    'luad': cptac.Luad(),
    'lscc': cptac.Lscc(),
    'ovarian': cptac.Ovarian()
}

cptac warning: The GBM dataset is under publication embargo until March 01, 2021. CPTAC is a community resource project and data are made available rapidly after generation for community research use. The embargo allows exploring and utilizing the data, but analysis may not be published until after the embargo date. Please see https://proteomics.cancer.gov/data-portal/about/data-use-agreement or enter cptac.embargo() to open the webpage for more details. (C:\Users\chels\anaconda3\envs\notebooks\lib\site-packages\ipykernel_launcher.py, line 6)


cptac warning: The HNSCC data is currently strictly reserved for CPTAC investigators. Otherwise, you are not authorized to access these data. Additionally, even after these data become publicly available, they will be subject to a publication embargo (see https://proteomics.cancer.gov/data-portal/about/data-use-agreement or enter cptac.embargo() to open the webpage for more details). (C:\Users\chels\anaconda3\envs\notebooks\lib\site-packages\ipykernel_launcher.py, line 7)


cptac warning: The LSCC data is currently strictly reserved for CPTAC investigators. Otherwise, you are not authorized to access these data. Additionally, even after these data become publicly available, they will be subject to a publication embargo (see https://proteomics.cancer.gov/data-portal/about/data-use-agreement or enter cptac.embargo() to open the webpage for more details). (C:\Users\chels\anaconda3\envs\notebooks\lib\site-packages\ipykernel_launcher.py, line 9)


In [4]:
dna_repair_genes = ['TP53', 'KMT2C', 'ATM', 'BAP1', 'BRCA2', 'POLQ', 'PRKDC', 'ATR', 'BRCA1', 'MSH6', 'POLE', 'FANCM', 'TP53BP1', 'CDK12', 'FANCD2', 'SLX4']

## Run Fisher Tests

In [5]:
repair_gene_dict = dict()
cancers = list()
genes = list()
pvalues = list()
odds_list = list()
for cancer in cancer_types.keys():
    has_event = pd.read_csv(f'{cancer}_has_event.tsv', sep='\t', index_col=0)
    mutations = cancer_types[cancer].get_somatic_mutation()
    drivers = mutations[mutations.Gene.isin(dna_repair_genes)]
    drivers['has_mutation'] = True
    driver_table = pd.pivot_table(drivers, index=drivers.index, values='has_mutation', columns=['Gene']).fillna(False)
    joined = has_event.join(driver_table).fillna(False)
    for gene in dna_repair_genes:
        if gene in joined.columns:
            event_table = pd.crosstab(joined['loss_event'], joined[gene])
            try:
                odds, pvalue = stats.fisher_exact(event_table)
                odds_list.append(odds)
                pvalues.append(pvalue)
                genes.append(gene)
                cancers.append(cancer)
            except:
                genes.append(gene)
                cancers.append(cancer)
                odds_list.append(None)
                pvalues.append(None)
results = pd.DataFrame({"gene": genes,
                        "cancer": cancers,
                        "pvalue": pvalues,
                        "odds": odds_list})

C:\Users\chels\anaconda3\envs\notebooks\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\chels\anaconda3\envs\notebooks\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\chels\anaconda3\envs\notebooks\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy o

In [6]:
results

,gene,cancer,pvalue,odds
0,TP53,brca,0.000076,13.783784
1,KMT2C,brca,0.591559,0.000000
2,ATM,brca,1.000000,0.000000
3,BRCA2,brca,1.000000,0.000000
4,POLQ,brca,0.455765,2.125000
...,...,...,...,...
120,FANCM,ovarian,1.000000,0.000000
121,TP53BP1,ovarian,0.570965,0.000000
122,CDK12,ovarian,1.000000,0.583333
123,FANCD2,ovarian,0.439394,3.083333


## Correct Pvalues

In [7]:
# Correct for Multiple Tests
corrected_data = pd.DataFrame()
for cancer in results.cancer.unique():
    cancer_results = results[results.cancer == cancer]
    pvalues = cancer_results.pvalue
    corrected = statsmodels.stats.multitest.multipletests(pvalues, method='fdr_bh')
    cancer_results['pvalues'] = corrected[1]
    if corrected_data.empty:
        corrected_data = cancer_results
    else:
        corrected_data = pd.concat([corrected_data, cancer_results])


C:\Users\chels\anaconda3\envs\notebooks\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\chels\anaconda3\envs\notebooks\lib\site-packages\statsmodels\stats\multitest.py:325: RuntimeWarning: invalid value encountered in less_equal
  reject = pvals_sorted <= ecdffactor*alpha
C:\Users\chels\anaconda3\envs\notebooks\lib\site-packages\statsmodels\stats\multitest.py:333: RuntimeWarning: invalid value encountered in greater
  pvals_corrected[pvals_corrected>1] = 1
C:\Users\chels\anaconda3\envs\notebooks\lib\site-packages\statsmodels\stats\multitest.py:256: RuntimeWarning: invalid value encountered in greater
  pvals_corrected[pvals_corrected>1] = 1


In [8]:
corrected_data[corrected_data.pvalues < 0.1 ]

,gene,cancer,pvalue,odds,pvalues
0,TP53,brca,0.000076,13.783784,0.000985
39,TP53,endo,0.000013,44.923077,0.000209
96,KMT2C,lscc,0.001751,9.428571,0.026259
